# Chapter 10: Advanced Data Fetching & Caching

Building on the data fetching fundamentals from Chapter 8, we now dive into advanced caching strategies and data fetching patterns. These powerful features enable you to build applications that are both incredibly fast and always up-to-date, providing the best possible user experience.

By the end of this chapter, you'll master advanced caching with Cache Components, implement sophisticated revalidation strategies, handle cache invalidation with tags, leverage Incremental Static Regeneration, and implement streaming data fetching for optimal performance.

## 10.1 Cache Components and "use cache" Directive

Cache Components provide granular control over caching behavior at the component level, allowing you to optimize performance while maintaining data freshness.

### Understanding "use cache" Directive

The `use cache` directive lets you specify caching behavior for specific components:

```tsx
// app/components/CachedUserProfile.tsx

interface UserProfile {
  id: string;
  name: string;
  email: string;
  avatar: string;
  bio: string;
}

// Server Component with cache directive
async function getUserProfile(userId: string): Promise<UserProfile> {
  const response = await fetch(`https://api.example.com/users/${userId}`, {
    next: { revalidate: 3600 }, // Cache for 1 hour
  });

  if (!response.ok) {
    throw new Error('Failed to fetch user profile');
  }

  return response.json();
}

export default async function CachedUserProfile({ 
  userId 
}: { 
  userId: string;
}) {
  const user = await getUserProfile(userId);

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <div className="flex items-start space-x-4">
        <div className="relative w-20 h-20 bg-gray-200 rounded-full overflow-hidden">
          <img
            src={user.avatar}
            alt={user.name}
            className="w-full h-full object-cover"
          />
        </div>
        
        <div className="flex-1">
          <h3 className="text-xl font-semibold text-gray-900 mb-1">
            {user.name}
          </h3>
          <p className="text-gray-600 mb-3">{user.email}</p>
          <p className="text-sm text-gray-500">{user.bio}</p>
        </div>
      </div>
    </div>
  );
}
```

### Nested Cache Components

Cache components can be nested for sophisticated caching strategies:

```tsx
// app/dashboard/page.tsx

// Cache configuration for different data types
async function getDashboardSummary() {
  const response = await fetch('https://api.example.com/dashboard/summary', {
    next: { revalidate: 300 }, // Cache for 5 minutes
  });
  return response.json();
}

async function getRecentActivity() {
  const response = await fetch('https://api.example.com/activity/recent', {
    cache: 'no-store', // Always fresh
  });
  return response.json();
}

async function getUserPreferences() {
  const response = await fetch('https://api.example.com/user/preferences', {
    next: { revalidate: 86400 }, // Cache for 24 hours
  });
  return response.json();
}

export default async function DashboardPage() {
  // Parallel data fetching with different caching strategies
  const [summary, activity, preferences] = await Promise.all([
    getDashboardSummary(),
    getRecentActivity(),
    getUserPreferences(),
  ]);

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Dashboard</h1>

      <div className="grid grid-cols-1 lg:grid-cols-3 gap-6">
        {/* Cached summary - updates every 5 minutes */}
        <div className="lg:col-span-2">
          <DashboardSummary summary={summary} />
        </div>

        {/* Always fresh activity feed */}
        <div>
          <RecentActivity activity={activity} />
        </div>
      </div>

      {/* Long-cached user preferences */}
      <div className="mt-6">
        <UserPreferences preferences={preferences} />
      </div>
    </div>
  );
}

// Server Component - inherits parent caching
async function DashboardSummary({ summary }: { summary: any }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Summary</h2>
      <div className="grid grid-cols-1 md:grid-cols-3 gap-4">
        <MetricCard label="Total Revenue" value={summary.revenue} />
        <MetricCard label="Active Users" value={summary.users} />
        <MetricCard label="Orders" value={summary.orders} />
      </div>
    </div>
  );
}

// Server Component - no caching
async function RecentActivity({ activity }: { activity: any[] }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Recent Activity</h2>
      <ul className="space-y-3">
        {activity.map((item) => (
          <li key={item.id} className="flex items-start space-x-3">
            <div className="w-2 h-2 bg-blue-500 rounded-full mt-2"></div>
            <div>
              <p className="text-sm font-medium text-gray-900">{item.action}</p>
              <p className="text-xs text-gray-500">{new Date(item.timestamp).toLocaleString()}</p>
            </div>
          </li>
        ))}
      </ul>
    </div>
  );
}

// Server Component - long cache
async function UserPreferences({ preferences }: { preferences: any }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Your Preferences</h2>
      <div className="space-y-3">
        <PreferenceItem label="Email Notifications" value={preferences.emailNotifications} />
        <PreferenceItem label="Theme" value={preferences.theme} />
        <PreferenceItem label="Language" value={preferences.language} />
      </div>
    </div>
  );
}

function MetricCard({ label, value }: { label: string; value: number }) {
  return (
    <div className="bg-gray-50 rounded-lg p-4">
      <p className="text-sm text-gray-600 mb-1">{label}</p>
      <p className="text-2xl font-bold text-gray-900">{value.toLocaleString()}</p>
    </div>
  );
}

function PreferenceItem({ label, value }: { label: string; value: string | boolean }) {
  return (
    <div className="flex justify-between items-center py-2 border-b border-gray-100">
      <span className="text-gray-700">{label}</span>
      <span className="font-medium text-gray-900">
        {typeof value === 'boolean' ? (value ? 'Enabled' : 'Disabled') : value}
      </span>
    </div>
  );
}
```

### Cache Component Patterns

Implement common caching patterns:

```tsx
// app/products/[id]/page.tsx

// Pattern 1: Critical data + supporting data
async function getProductCriticalData(productId: string) {
  const response = await fetch(`https://api.example.com/products/${productId}`, {
    next: { revalidate: 3600 }, // Cache for 1 hour
  });
  return response.json();
}

async function getProductReviews(productId: string) {
  const response = await fetch(`https://api.example.com/products/${productId}/reviews`, {
    next: { revalidate: 300 }, // Cache for 5 minutes
  });
  return response.json();
}

async function getProductRelatedProducts(productId: string) {
  const response = await fetch(`https://api.example.com/products/${productId}/related`, {
    next: { revalidate: 1800 }, // Cache for 30 minutes
  });
  return response.json();
}

export default async function ProductPage({ params }: { params: { id: string } }) {
  // Fetch critical data first
  const product = await getProductCriticalData(params.id);

  return (
    <div className="container mx-auto px-4 py-8">
      <ProductDetails product={product}>
        {/* Nested components with different caching */}
        <Suspense fallback={<ReviewsLoadingSkeleton />}>
          <ProductReviews productId={params.id} />
        </Suspense>
        
        <Suspense fallback={<RelatedLoadingSkeleton />}>
          <RelatedProducts productId={params.id} />
        </Suspense>
      </ProductDetails>
    </div>
  );
}

// Product details - uses parent cache
function ProductDetails({ product, children }: { product: any; children: React.ReactNode }) {
  return (
    <div className="space-y-8">
      <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
        <div className="aspect-[4/3] bg-gray-100 rounded-lg overflow-hidden">
          <img
            src={product.imageUrl}
            alt={product.name}
            className="w-full h-full object-cover"
          />
        </div>
        
        <div>
          <h1 className="text-3xl font-bold text-gray-900 mb-4">
            {product.name}
          </h1>
          <p className="text-gray-600 mb-6">{product.description}</p>
          <p className="text-3xl font-bold text-blue-600 mb-6">
            ${product.price.toFixed(2)}
          </p>
          <button className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 transition-colors">
            Add to Cart
          </button>
        </div>
      </div>

      {/* Children load independently */}
      <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
        {children}
      </div>
    </div>
  );
}

// Reviews with independent cache
async function ProductReviews({ productId }: { productId: string }) {
  const reviews = await getProductReviews(productId);

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-6">Reviews</h2>
      
      {reviews.length === 0 ? (
        <p className="text-gray-600">No reviews yet.</p>
      ) : (
        <div className="space-y-6">
          {reviews.map((review: any) => (
            <div key={review.id} className="border-b border-gray-200 pb-6 last:border-0">
              <div className="flex items-start space-x-4">
                <div className="w-10 h-10 bg-gray-200 rounded-full flex items-center justify-center">
                  {review.author[0]}
                </div>
                <div className="flex-1">
                  <div className="flex items-center justify-between mb-2">
                    <h3 className="font-semibold text-gray-900">{review.author}</h3>
                    <div className="flex items-center space-x-1">
                      {[...Array(5)].map((_, i) => (
                        <Star key={i} filled={i < review.rating} />
                      ))}
                    </div>
                  </div>
                  <p className="text-gray-600">{review.comment}</p>
                  <p className="text-sm text-gray-500 mt-2">
                    {new Date(review.date).toLocaleDateString()}
                  </p>
                </div>
              </div>
            </div>
          ))}
        </div>
      )}
    </div>
  );
}

// Related products with independent cache
async function RelatedProducts({ productId }: { productId: string }) {
  const products = await getProductRelatedProducts(productId);

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-6">Related Products</h2>
      
      <div className="grid grid-cols-1 sm:grid-cols-2 gap-4">
        {products.map((product: any) => (
          <a
            key={product.id}
            href={`/products/${product.id}`}
            className="group block"
          >
            <div className="aspect-square bg-gray-100 rounded-lg overflow-hidden mb-3">
              <img
                src={product.thumbnailUrl}
                alt={product.name}
                className="w-full h-full object-cover group-hover:scale-105 transition-transform"
              />
            </div>
            <h3 className="font-semibold text-gray-900 mb-1 group-hover:text-blue-600">
              {product.name}
            </h3>
            <p className="text-gray-600">${product.price.toFixed(2)}</p>
          </a>
        ))}
      </div>
    </div>
  );
}

function Star({ filled }: { filled: boolean }) {
  return (
    <svg
      className={`w-4 h-4 ${filled ? 'text-yellow-400' : 'text-gray-300'}`}
      fill="currentColor"
      viewBox="0 0 20 20"
    >
      <path d="M9.049 2.927c.3-.921 1.603-.921 1.902 0l1.07 3.292a1 1 0 00.95.69h3.462c.969 0 1.371 1.24.588 1.81l-2.8 2.034a1 1 0 00-.364 1.118l1.07 3.292c.3.921-.755 1.688-1.54 1.118l-2.8-2.034a1 1 0 00-1.175 0l-2.8 2.034c-.784.57-1.838-.197-1.539-1.118l1.07-3.292a1 1 0 00-.364-1.118L2.98 8.72c-.783-.57-.38-1.81.588-1.81h3.461a1 1 0 00.951-.69l1.07-3.292z" />
    </svg>
  );
}

function ReviewsLoadingSkeleton() {
  return (
    <div className="bg-white rounded-lg shadow-md p-6 animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/4 mb-6"></div>
      <div className="space-y-6">
        {[1, 2, 3].map((i) => (
          <div key={i} className="space-y-3">
            <div className="flex items-center space-x-4">
              <div className="w-10 h-10 bg-gray-200 rounded-full"></div>
              <div className="flex-1 space-y-2">
                <div className="h-4 bg-gray-200 rounded w-1/3"></div>
                <div className="h-3 bg-gray-200 rounded w-1/2"></div>
              </div>
            </div>
            <div className="h-16 bg-gray-200 rounded"></div>
          </div>
        ))}
      </div>
    </div>
  );
}

function RelatedLoadingSkeleton() {
  return (
    <div className="bg-white rounded-lg shadow-md p-6 animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/4 mb-6"></div>
      <div className="grid grid-cols-1 sm:grid-cols-2 gap-4">
        {[1, 2, 3, 4].map((i) => (
          <div key={i} className="space-y-3">
            <div className="aspect-square bg-gray-200 rounded"></div>
            <div className="h-4 bg-gray-200 rounded w-3/4"></div>
            <div className="h-3 bg-gray-200 rounded w-1/2"></div>
          </div>
        ))}
      </div>
    </div>
  );
}
```

## 10.2 Revalidation Strategies

Revalidation determines when and how cached data is updated. Choosing the right strategy is crucial for balancing performance and data freshness.

### Time-Based Revalidation

Automatically revalidate data at specific intervals:

```tsx
// lib/api.ts

// Short-lived cache for frequently changing data
export async function getStockPrices() {
  const response = await fetch('https://api.example.com/stocks', {
    next: { revalidate: 60 }, // Revalidate every minute
  });
  return response.json();
}

// Medium-lived cache for moderately changing data
export async function getProductInventory() {
  const response = await fetch('https://api.example.com/inventory', {
    next: { revalidate: 300 }, // Revalidate every 5 minutes
  });
  return response.json();
}

// Long-lived cache for rarely changing data
export async function getCompanyInfo() {
  const response = await fetch('https://api.example.com/company', {
    next: { revalidate: 86400 }, // Revalidate every day
  });
  return response.json();
}
```

### Stale-While-Revalidate

Serve stale data while revalidating in the background:

```tsx
// app/dashboard/analytics/page.tsx

async function getAnalyticsData() {
  const response = await fetch('https://api.example.com/analytics', {
    next: { 
      revalidate: 300, // Revalidate every 5 minutes
    },
  });

  if (!response.ok) {
    throw new Error('Failed to fetch analytics data');
  }

  return response.json();
}

export default async function AnalyticsPage() {
  const data = await getAnalyticsData();

  const lastUpdated = new Date().toLocaleTimeString();

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="flex justify-between items-center mb-8">
        <h1 className="text-3xl font-bold text-gray-900">Analytics</h1>
        <div className="text-sm text-gray-500">
          Last updated: {lastUpdated}
          <span className="ml-2 text-blue-600">
            (Updates every 5 minutes)
          </span>
        </div>
      </div>

      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-6">
        <AnalyticsCard 
          title="Revenue" 
          value={data.revenue} 
          change={data.revenueChange}
          positive={data.revenueChange >= 0}
        />
        <AnalyticsCard 
          title="Users" 
          value={data.users} 
          change={data.usersChange}
          positive={data.usersChange >= 0}
        />
        <AnalyticsCard 
          title="Sessions" 
          value={data.sessions} 
          change={data.sessionsChange}
          positive={data.sessionsChange >= 0}
        />
        <AnalyticsCard 
          title="Conversion" 
          value={`${data.conversion}%`} 
          change={`${data.conversionChange}%`}
          positive={data.conversionChange >= 0}
        />
      </div>

      <div className="mt-8 grid grid-cols-1 lg:grid-cols-2 gap-6">
        <ChartCard title="Revenue Over Time" data={data.revenueChart} />
        <ChartCard title="User Growth" data={data.userChart} />
      </div>
    </div>
  );
}

function AnalyticsCard({ 
  title, 
  value, 
  change, 
  positive 
}: { 
  title: string; 
  value: string | number; 
  change: string | number; 
  positive: boolean;
}) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h3 className="text-sm font-medium text-gray-600 mb-2">{title}</h3>
      <p className="text-2xl font-bold text-gray-900 mb-2">{value}</p>
      <p className={`text-sm ${positive ? 'text-green-600' : 'text-red-600'}`}>
        {positive ? '+' : ''}{change} from last period
      </p>
    </div>
  );
}

function ChartCard({ title, data }: { title: string; data: any[] }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h3 className="text-lg font-semibold text-gray-900 mb-4">{title}</h3>
      <div className="h-64 bg-gray-100 rounded flex items-center justify-center">
        <p className="text-gray-600">Chart visualization would go here</p>
      </div>
    </div>
  );
}
```

### On-Demand Revalidation

Revalidate data when specific events occur:

```tsx
// app/actions/revalidate.ts
'use server';

import { revalidatePath } from 'next/cache';

// Revalidate after creating content
export async function revalidateAfterCreate(slug: string) {
  revalidatePath(`/blog/${slug}`);
  revalidatePath('/blog'); // Also revalidate blog list
  return { success: true };
}

// Revalidate after updating content
export async function revalidateAfterUpdate(slug: string) {
  revalidatePath(`/blog/${slug}`);
  revalidatePath('/blog');
  return { success: true };
}

// Revalidate after deleting content
export async function revalidateAfterDelete(slug: string) {
  revalidatePath(`/blog/${slug}`);
  revalidatePath('/blog');
  return { success: true };
}
```

## 10.3 On-Demand Revalidation

On-demand revalidation gives you precise control over when cached data is updated, based on application events or triggers.

### Revalidating on Form Submission

```tsx
// app/blog/new/page.tsx
"use client";

import { useState } from 'react';
import { useRouter } from 'next/navigation';
import { revalidateAfterCreate } from '@/app/actions/revalidate';

interface FormData {
  title: string;
  content: string;
  category: string;
  tags: string[];
  published: boolean;
}

export default function NewBlogPost() {
  const router = useRouter();
  const [formData, setFormData] = useState<FormData>({
    title: '',
    content: '',
    category: '',
    tags: [],
    published: false,
  });
  const [isSubmitting, setIsSubmitting] = useState(false);
  const [submitError, setSubmitError] = useState<string | null>(null);

  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault();
    setIsSubmitting(true);
    setSubmitError(null);

    try {
      // Create blog post
      const response = await fetch('/api/blog/posts', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify(formData),
      });

      if (!response.ok) {
        throw new Error('Failed to create blog post');
      }

      const result = await response.json();
      const slug = result.slug;

      // Revalidate cached pages
      await revalidateAfterCreate(slug);

      // Redirect to new post
      router.push(`/blog/${slug}`);
    } catch (error) {
      setSubmitError(error instanceof Error ? error.message : 'An error occurred');
      setIsSubmitting(false);
    }
  };

  return (
    <div className="container mx-auto px-4 py-8 max-w-3xl">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Create New Blog Post</h1>

      {submitError && (
        <div className="bg-red-50 border border-red-200 rounded-lg p-4 mb-6">
          <p className="text-red-700">{submitError}</p>
        </div>
      )}

      <form onSubmit={handleSubmit} className="space-y-6">
        <div>
          <label htmlFor="title" className="block text-sm font-medium text-gray-700 mb-2">
            Title
          </label>
          <input
            type="text"
            id="title"
            value={formData.title}
            onChange={(e) => setFormData({ ...formData, title: e.target.value })}
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
            required
          />
        </div>

        <div>
          <label htmlFor="category" className="block text-sm font-medium text-gray-700 mb-2">
            Category
          </label>
          <select
            id="category"
            value={formData.category}
            onChange={(e) => setFormData({ ...formData, category: e.target.value })}
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
            required
          >
            <option value="">Select a category</option>
            <option value="technology">Technology</option>
            <option value="design">Design</option>
            <option value="business">Business</option>
            <option value="lifestyle">Lifestyle</option>
          </select>
        </div>

        <div>
          <label htmlFor="content" className="block text-sm font-medium text-gray-700 mb-2">
            Content
          </label>
          <textarea
            id="content"
            value={formData.content}
            onChange={(e) => setFormData({ ...formData, content: e.target.value })}
            rows={10}
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
            required
          />
        </div>

        <div className="flex items-center space-x-4">
          <label className="flex items-center">
            <input
              type="checkbox"
              checked={formData.published}
              onChange={(e) => setFormData({ ...formData, published: e.target.checked })}
              className="mr-2"
            />
            <span className="text-gray-700">Publish immediately</span>
          </label>
        </div>

        <div className="flex space-x-4">
          <button
            type="submit"
            disabled={isSubmitting}
            className="flex-1 bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 disabled:opacity-50 disabled:cursor-not-allowed transition-colors"
          >
            {isSubmitting ? 'Creating...' : 'Create Post'}
          </button>
          
          <button
            type="button"
            onClick={() => router.back()}
            className="px-6 py-3 bg-gray-200 text-gray-900 rounded-lg hover:bg-gray-300 transition-colors"
          >
            Cancel
          </button>
        </div>
      </form>
    </div>
  );
}
```

### Revalidating on External Events

```tsx
// app/api/webhooks/stripe/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { revalidateTag } from 'next/cache';

export async function POST(request: NextRequest) {
  try {
    const body = await request.json();
    const eventType = body.type;

    switch (eventType) {
      case 'checkout.session.completed':
        // Revalidate user's orders page
        revalidateTag(`user-orders-${body.data.object.metadata.userId}`);
        
        // Revalidate product inventory
        body.data.object.line_items.data.forEach((item: any) => {
          revalidateTag(`product-${item.price.product}`);
          revalidateTag('products-inventory');
        });
        
        break;

      case 'customer.subscription.created':
      case 'customer.subscription.updated':
      case 'customer.subscription.deleted':
        // Revalidate user's subscription status
        revalidateTag(`user-subscription-${body.data.object.customer}`);
        
        break;

      default:
        console.log(`Unhandled event type: ${eventType}`);
    }

    return NextResponse.json({ received: true });
  } catch (error) {
    console.error('Webhook error:', error);
    return NextResponse.json(
      { error: 'Webhook handler failed' },
      { status: 500 }
    );
  }
}
```

### Scheduled Revalidation

Set up scheduled tasks to revalidate data:

```tsx
// lib/scheduled-revalidation.ts

import { revalidatePath, revalidateTag } from 'next/cache';

// Revalidate daily data
export async function revalidateDailyData() {
  console.log('Revalidating daily data...');
  
  try {
    // Revalidate daily analytics
    revalidatePath('/dashboard/analytics');
    
    // Revalidate product inventory
    revalidateTag('products-inventory');
    
    // Revalidate exchange rates
    revalidateTag('exchange-rates');
    
    console.log('Daily revalidation completed');
    return { success: true };
  } catch (error) {
    console.error('Daily revalidation failed:', error);
    return { success: false, error };
  }
}

// Revalidate hourly data
export async function revalidateHourlyData() {
  console.log('Revalidating hourly data...');
  
  try {
    // Revalidate stock prices
    revalidateTag('stock-prices');
    
    // Revalidate weather data
    revalidateTag('weather-data');
    
    console.log('Hourly revalidation completed');
    return { success: true };
  } catch (error) {
    console.error('Hourly revalidation failed:', error);
    return { success: false, error };
  }
}
```

## 10.4 Tag-Based Cache Invalidation

Tags provide a powerful way to organize and invalidate related cached data across your application.

### Tagging Data Fetches

Add tags to your data fetches for organized invalidation:

```tsx
// lib/api-cache.ts

// Product-related fetches
export async function getProduct(productId: string) {
  const response = await fetch(`https://api.example.com/products/${productId}`, {
    next: { 
      tags: [`product-${productId}`, 'products'],
      revalidate: 3600,
    },
  });
  return response.json();
}

export async function getProductList(category?: string) {
  const url = category 
    ? `https://api.example.com/products?category=${category}`
    : 'https://api.example.com/products';
  
  const response = await fetch(url, {
    next: { 
      tags: category 
        ? [`products-category-${category}`, 'products']
        : ['products'],
      revalidate: 300,
    },
  });
  return response.json();
}

export async function getProductReviews(productId: string) {
  const response = await fetch(`https://api.example.com/products/${productId}/reviews`, {
    next: { 
      tags: [`product-reviews-${productId}`, 'reviews'],
      revalidate: 600,
    },
  });
  return response.json();
}

// User-related fetches
export async function getUserProfile(userId: string) {
  const response = await fetch(`https://api.example.com/users/${userId}`, {
    next: { 
      tags: [`user-${userId}`, 'users'],
      revalidate: 1800,
    },
  });
  return response.json();
}

export async function getUserOrders(userId: string) {
  const response = await fetch(`https://api.example.com/users/${userId}/orders`, {
    next: { 
      tags: [`user-orders-${userId}`, 'orders'],
      revalidate: 300,
    },
  });
  return response.json();
}

// Content-related fetches
export async function getBlogPost(slug: string) {
  const response = await fetch(`https://api.example.com/blog/${slug}`, {
    next: { 
      tags: [`blog-post-${slug}`, 'blog-posts'],
      revalidate: 3600,
    },
  });
  return response.json();
}

export async function getBlogPosts(category?: string) {
  const url = category 
    ? `https://api.example.com/blog?category=${category}`
    : 'https://api.example.com/blog';
  
  const response = await fetch(url, {
    next: { 
      tags: category 
        ? [`blog-category-${category}`, 'blog-posts']
        : ['blog-posts'],
      revalidate: 600,
    },
  });
  return response.json();
}
```

### Invalidating by Tag

Revalidate data using tags:

```tsx
// app/actions/invalidate-cache.ts
'use server';

import { revalidateTag } from 'next/cache';

// Invalidate specific product
export async function invalidateProduct(productId: string) {
  revalidateTag(`product-${productId}`);
  revalidateTag('products');
  return { success: true };
}

// Invalidate product category
export async function invalidateProductCategory(categoryId: string) {
  revalidateTag(`products-category-${categoryId}`);
  revalidateTag('products');
  return { success: true };
}

// Invalidate all products
export async function invalidateAllProducts() {
  revalidateTag('products');
  return { success: true };
}

// Invalidate user data
export async function invalidateUser(userId: string) {
  revalidateTag(`user-${userId}`);
  revalidateTag('users');
  return { success: true };
}

// Invalidate user orders
export async function invalidateUserOrders(userId: string) {
  revalidateTag(`user-orders-${userId}`);
  revalidateTag('orders');
  return { success: true };
}

// Invalidate blog post
export async function invalidateBlogPost(slug: string) {
  revalidateTag(`blog-post-${slug}`);
  revalidateTag('blog-posts');
  return { success: true };
}

// Invalidate blog category
export async function invalidateBlogCategory(category: string) {
  revalidateTag(`blog-category-${category}`);
  revalidateTag('blog-posts');
  return { success: true };
}
```

### Using Tag-Based Invalidation in Actions

```tsx
// app/actions/product.ts
'use server';

import { revalidateTag } from 'next/cache';
import { getProduct, getProductList } from '@/lib/api-cache';
import { invalidateProduct, invalidateProductCategory } from '@/app/actions/invalidate-cache';

// Create new product
export async function createProduct(formData: FormData) {
  try {
    const response = await fetch('https://api.example.com/products', {
      method: 'POST',
      body: formData,
    });

    if (!response.ok) {
      throw new Error('Failed to create product');
    }

    const product = await response.json();

    // Invalidate product caches
    revalidateTag('products');
    if (product.categoryId) {
      revalidateTag(`products-category-${product.categoryId}`);
    }

    return { success: true, product };
  } catch (error) {
    console.error('Error creating product:', error);
    return { success: false, error: 'Failed to create product' };
  }
}

// Update product
export async function updateProduct(productId: string, formData: FormData) {
  try {
    const response = await fetch(`https://api.example.com/products/${productId}`, {
      method: 'PATCH',
      body: formData,
    });

    if (!response.ok) {
      throw new Error('Failed to update product');
    }

    const product = await response.json();

    // Invalidate specific product and related caches
    await invalidateProduct(productId);
    
    // Also invalidate category if category changed
    const oldProduct = await getProduct(productId);
    if (oldProduct.categoryId !== product.categoryId) {
      await invalidateProductCategory(oldProduct.categoryId);
      await invalidateProductCategory(product.categoryId);
    }

    return { success: true, product };
  } catch (error) {
    console.error('Error updating product:', error);
    return { success: false, error: 'Failed to update product' };
  }
}

// Delete product
export async function deleteProduct(productId: string) {
  try {
    const response = await fetch(`https://api.example.com/products/${productId}`, {
      method: 'DELETE',
    });

    if (!response.ok) {
      throw new Error('Failed to delete product');
    }

    const product = await response.json();

    // Invalidate product and category caches
    await invalidateProduct(productId);
    if (product.categoryId) {
      await invalidateProductCategory(product.categoryId);
    }

    return { success: true };
  } catch (error) {
    console.error('Error deleting product:', error);
    return { success: false, error: 'Failed to delete product' };
  }
}
```

## 10.5 Incremental Static Regeneration (ISR)

ISR allows you to update static pages after you've built your site, combining the benefits of static generation with dynamic content updates.

### Basic ISR Setup

Set up ISR with `revalidate` option:

```tsx
// app/blog/[slug]/page.tsx

interface BlogPost {
  id: string;
  slug: string;
  title: string;
  content: string;
  author: {
    name: string;
    avatar: string;
  };
  category: string;
  tags: string[];
  publishedAt: string;
  updatedAt: string;
}

async function getBlogPost(slug: string): Promise<BlogPost> {
  const response = await fetch(`https://api.example.com/blog/${slug}`, {
    next: { 
      revalidate: 3600, // Regenerate every hour
      tags: [`blog-post-${slug}`, 'blog-posts'],
    },
  });

  if (!response.ok) {
    throw new Error('Failed to fetch blog post');
  }

  return response.json();
}

export async function generateStaticParams() {
  const posts = await fetch('https://api.example.com/blog').then(res => res.json());
  
  return posts.map((post: BlogPost) => ({
    slug: post.slug,
  }));
}

export default async function BlogPostPage({ params }: { params: { slug: string } }) {
  const post = await getBlogPost(params.slug);

  return (
    <article className="max-w-4xl mx-auto px-4 py-8">
      {/* Header */}
      <header className="mb-8">
        <div className="flex items-center space-x-2 text-sm text-gray-600 mb-4">
          <a href={`/blog/category/${post.category}`} className="hover:text-blue-600">
            {post.category}
          </a>
          <span>•</span>
          <time>{new Date(post.publishedAt).toLocaleDateString()}</time>
        </div>
        
        <h1 className="text-4xl font-bold text-gray-900 mb-4">
          {post.title}
        </h1>
        
        <div className="flex items-center space-x-4">
          <div className="relative w-12 h-12 bg-gray-200 rounded-full overflow-hidden">
            <img
              src={post.author.avatar}
              alt={post.author.name}
              className="w-full h-full object-cover"
            />
          </div>
          <div>
            <p className="font-medium text-gray-900">{post.author.name}</p>
            <p className="text-sm text-gray-600">Author</p>
          </div>
        </div>
      </header>

      {/* Content */}
      <div className="prose prose-lg max-w-none">
        <div dangerouslySetInnerHTML={{ __html: post.content }} />
      </div>

      {/* Tags */}
      <div className="mt-8 pt-8 border-t border-gray-200">
        <h3 className="text-sm font-medium text-gray-600 mb-3">Tags</h3>
        <div className="flex flex-wrap gap-2">
          {post.tags.map((tag) => (
            <a
              key={tag}
              href={`/blog/tag/${tag}`}
              className="px-3 py-1 bg-gray-100 text-gray-700 rounded-full text-sm hover:bg-gray-200"
            >
              {tag}
            </a>
          ))}
        </div>
      </div>

      {/* Last updated */}
      <div className="mt-8 text-sm text-gray-500">
        Last updated: {new Date(post.updatedAt).toLocaleString()}
      </div>
    </article>
  );
}
```

### On-Demand ISR

Regenerate pages on-demand:

```tsx
// app/actions/regenerate.ts
'use server';

import { revalidatePath } from 'next/cache';

// Regenerate specific blog post
export async function regenerateBlogPost(slug: string) {
  revalidatePath(`/blog/${slug}`);
  return { success: true };
}

// Regenerate all blog posts
export async function regenerateAllBlogPosts() {
  revalidatePath('/blog');
  return { success: true };
}

// Regenerate blog category
export async function regenerateBlogCategory(category: string) {
  revalidatePath(`/blog/category/${category}`);
  return { success: true };
}
```

### ISR with Fallback

Handle ISR regeneration with fallback content:

```tsx
// app/products/[id]/page.tsx

interface Product {
  id: string;
  name: string;
  description: string;
  price: number;
  imageUrl: string;
  inStock: boolean;
  category: string;
}

async function getProduct(id: string): Promise<Product | null> {
  try {
    const response = await fetch(`https://api.example.com/products/${id}`, {
      next: { 
        revalidate: 3600, // Regenerate every hour
      },
    });

    if (response.status === 404) {
      return null;
    }

    if (!response.ok) {
      throw new Error('Failed to fetch product');
    }

    return response.json();
  } catch (error) {
    console.error('Error fetching product:', error);
    return null; // Return null to trigger fallback
  }
}

export async function generateStaticParams() {
  const products = await fetch('https://api.example.com/products').then(res => res.json());
  
  return products.slice(0, 100).map((product: Product) => ({
    id: product.id,
  }));
}

export default async function ProductPage({ params }: { params: { id: string } }) {
  const product = await getProduct(params.id);

  // If product is null, show fallback page
  if (!product) {
    return (
      <div className="container mx-auto px-4 py-8">
        <div className="max-w-md mx-auto text-center">
          <div className="mb-6">
            <svg className="w-16 h-16 mx-auto text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
              <path strokeLinecap="round" strokeLinejoin="round" strokeWidth={2} d="M9.172 16.172a4 4 0 015.656 0M9 10h.01M15 10h.01M21 12a9 9 0 11-18 0 9 9 0 0118 0z" />
            </svg>
          </div>
          
          <h1 className="text-2xl font-bold text-gray-900 mb-4">
            Product Not Found
          </h1>
          
          <p className="text-gray-600 mb-6">
            The product you're looking for is not available at the moment. 
            It may be out of stock or temporarily unavailable.
          </p>
          
          <a
            href="/products"
            className="inline-block bg-blue-600 text-white px-6 py-3 rounded-lg hover:bg-blue-700 transition-colors"
          >
            Browse Other Products
          </a>
          
          <p className="text-sm text-gray-500 mt-4">
            This page will be updated automatically when the product becomes available.
          </p>
        </div>
      </div>
    );
  }

  // Show product page
  return (
    <div className="container mx-auto px-4 py-8">
      <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
        <div className="aspect-[4/3] bg-gray-100 rounded-lg overflow-hidden">
          <img
            src={product.imageUrl}
            alt={product.name}
            className="w-full h-full object-cover"
          />
        </div>
        
        <div>
          <h1 className="text-3xl font-bold text-gray-900 mb-4">
            {product.name}
          </h1>
          
          <p className="text-gray-600 mb-6">{product.description}</p>
          
          <div className="mb-6">
            <p className="text-3xl font-bold text-blue-600">
              ${product.price.toFixed(2)}
            </p>
          </div>
          
          <div className="mb-6">
            <span className={`inline-block px-3 py-1 rounded-full text-sm ${
              product.inStock 
                ? 'bg-green-100 text-green-800' 
                : 'bg-red-100 text-red-800'
            }`}>
              {product.inStock ? 'In Stock' : 'Out of Stock'}
            </span>
          </div>
          
          <button
            disabled={!product.inStock}
            className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 disabled:opacity-50 disabled:cursor-not-allowed transition-colors"
          >
            {product.inStock ? 'Add to Cart' : 'Out of Stock'}
          </button>
        </div>
      </div>
    </div>
  );
}
```

## 10.6 Streaming with Suspense

Streaming allows you to progressively render parts of your page as data becomes available, improving perceived performance and user experience.

### Basic Streaming with Suspense

Stream different sections of your page:

```tsx
// app/dashboard/page.tsx

import { Suspense } from 'react';

// Slow component (simulates long data fetch)
async function SlowComponent() {
  await new Promise(resolve => setTimeout(resolve, 3000));
  
  const data = await fetch('https://api.example.com/slow-data').then(res => res.json());
  
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Slow Data</h2>
      <pre className="bg-gray-100 p-4 rounded overflow-auto">{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}

// Fast component
async function FastComponent() {
  const data = await fetch('https://api.example.com/fast-data').then(res => res.json());
  
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Fast Data</h2>
      <pre className="bg-gray-100 p-4 rounded overflow-auto">{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}

// Medium component
async function MediumComponent() {
  await new Promise(resolve => setTimeout(resolve, 1500));
  
  const data = await fetch('https://api.example.com/medium-data').then(res => res.json());
  
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Medium Data</h2>
      <pre className="bg-gray-100 p-4 rounded overflow-auto">{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}

export default function DashboardPage() {
  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Dashboard</h1>

      <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
        {/* Fast component loads immediately */}
        <FastComponent />

        {/* Medium component shows skeleton then loads */}
        <Suspense fallback={<SkeletonCard />}>
          <MediumComponent />
        </Suspense>

        {/* Slow component shows skeleton then loads */}
        <Suspense fallback={<SkeletonCard />}>
          <SlowComponent />
        </Suspense>
      </div>
    </div>
  );
}

function SkeletonCard() {
  return (
    <div className="bg-white rounded-lg shadow-md p-6 animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/3 mb-4"></div>
      <div className="space-y-2">
        <div className="h-4 bg-gray-200 rounded"></div>
        <div className="h-4 bg-gray-200 rounded w-5/6"></div>
        <div className="h-4 bg-gray-200 rounded w-4/6"></div>
      </div>
    </div>
  );
}
```

### Streaming Lists with Suspense

Stream list items as they become available:

```tsx
// app/users/page.tsx

import { Suspense } from 'react';

interface User {
  id: string;
  name: string;
  email: string;
  role: string;
  avatar: string;
}

// Fetch users in chunks
async function getUsersInChunks() {
  const allUsers: User[] = [];
  
  // Simulate fetching users in chunks
  for (let i = 0; i < 3; i++) {
    await new Promise(resolve => setTimeout(resolve, 1000));
    const chunk = await fetch(`https://api.example.com/users?offset=${i * 10}&limit=10`)
      .then(res => res.json());
    allUsers.push(...chunk);
  }
  
  return allUsers;
}

async function UserList() {
  const users = await getUsersInChunks();
  
  return (
    <div className="space-y-3">
      {users.map((user) => (
        <UserCard key={user.id} user={user} />
      ))}
    </div>
  );
}

function UserCard({ user }: { user: User }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-4 flex items-center space-x-4">
      <div className="relative w-12 h-12 bg-gray-200 rounded-full overflow-hidden">
        <img
          src={user.avatar}
          alt={user.name}
          className="w-full h-full object-cover"
        />
      </div>
      <div className="flex-1">
        <h3 className="font-semibold text-gray-900">{user.name}</h3>
        <p className="text-sm text-gray-600">{user.email}</p>
      </div>
      <span className="px-3 py-1 bg-blue-100 text-blue-800 rounded-full text-sm">
        {user.role}
      </span>
    </div>
  );
}

function UserCardSkeleton() {
  return (
    <div className="bg-white rounded-lg shadow-md p-4 flex items-center space-x-4 animate-pulse">
      <div className="w-12 h-12 bg-gray-200 rounded-full"></div>
      <div className="flex-1 space-y-2">
        <div className="h-4 bg-gray-200 rounded w-1/3"></div>
        <div className="h-3 bg-gray-200 rounded w-1/2"></div>
      </div>
      <div className="h-6 bg-gray-200 rounded w-16"></div>
    </div>
  );
}

export default function UsersPage() {
  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Users</h1>

      <Suspense
        fallback={
          <div className="space-y-3">
            {[1, 2, 3, 4, 5].map((i) => (
              <UserCardSkeleton key={i} />
            ))}
          </div>
        }
      >
        <UserList />
      </Suspense>
    </div>
  );
}
```

### Streaming with Error Boundaries

Combine streaming with error handling:

```tsx
// app/dashboard/analytics/page.tsx

import { Suspense } from 'react';

// Component that might fail
async function RiskyComponent() {
  await new Promise(resolve => setTimeout(resolve, 2000));
  
  const response = await fetch('https://api.example.com/risky-data');
  
  if (!response.ok) {
    throw new Error('Failed to fetch risky data');
  }
  
  const data = await response.json();
  
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Risky Data</h2>
      <pre className="bg-gray-100 p-4 rounded overflow-auto">{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}

// Safe component
async function SafeComponent() {
  const data = await fetch('https://api.example.com/safe-data').then(res => res.json());
  
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">Safe Data</h2>
      <pre className="bg-gray-100 p-4 rounded overflow-auto">{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}

// Error boundary component
function AnalyticsError({ error, reset }: { error: Error; reset: () => void }) {
  return (
    <div className="bg-red-50 border border-red-200 rounded-lg p-6">
      <h2 className="text-xl font-semibold text-red-900 mb-2">Error Loading Data</h2>
      <p className="text-red-700 mb-4">{error.message}</p>
      <button
        onClick={reset}
        className="px-4 py-2 bg-red-600 text-white rounded hover:bg-red-700"
      >
        Retry
      </button>
    </div>
  );
}

export default function AnalyticsPage() {
  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Analytics</h1>

      <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
        {/* Safe component with simple loading */}
        <SafeComponent />

        {/* Risky component with error boundary */}
        <Suspense fallback={<SkeletonCard />}>
          <RiskyComponent />
        </Suspense>
      </div>
    </div>
  );
}

function SkeletonCard() {
  return (
    <div className="bg-white rounded-lg shadow-md p-6 animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/3 mb-4"></div>
      <div className="space-y-2">
        <div className="h-4 bg-gray-200 rounded"></div>
        <div className="h-4 bg-gray-200 rounded w-5/6"></div>
      </div>
    </div>
  );
}
```

## 10.7 Parallel and Sequential Data Fetching

Choosing between parallel and sequential data fetching significantly impacts performance. Understanding when to use each approach is crucial for building responsive applications.

### Parallel Data Fetching

Fetch independent data in parallel for optimal performance:

```tsx
// app/dashboard/page.tsx

// Fetch independent data sources in parallel
async function getDashboardData() {
  const [
    users,
    posts,
    comments,
    analytics,
  ] = await Promise.all([
    fetch('https://api.example.com/users').then(res => res.json()),
    fetch('https://api.example.com/posts').then(res => res.json()),
    fetch('https://api.example.com/comments').then(res => res.json()),
    fetch('https://api.example.com/analytics').then(res => res.json()),
  ]);

  return { users, posts, comments, analytics };
}

export default async function DashboardPage() {
  const data = await getDashboardData();

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Dashboard</h1>

      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-6">
        <DashboardCard 
          title="Total Users" 
          value={data.users.length} 
          icon="👥"
          color="blue"
        />
        <DashboardCard 
          title="Total Posts" 
          value={data.posts.length} 
          icon="📝"
          color="green"
        />
        <DashboardCard 
          title="Total Comments" 
          value={data.comments.length} 
          icon="💬"
          color="purple"
        />
        <DashboardCard 
          title="Analytics Score" 
          value={data.analytics.score} 
          icon="📊"
          color="orange"
        />
      </div>
    </div>
  );
}

function DashboardCard({ 
  title, 
  value, 
  icon, 
  color 
}: { 
  title: string; 
  value: number | string; 
  icon: string; 
  color: string;
}) {
  const colorClasses = {
    blue: 'bg-blue-100 text-blue-800',
    green: 'bg-green-100 text-green-800',
    purple: 'bg-purple-100 text-purple-800',
    orange: 'bg-orange-100 text-orange-800',
  };

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <div className="flex items-center justify-between mb-4">
        <span className="text-2xl">{icon}</span>
        <span className={`px-2 py-1 rounded-full text-xs font-medium ${colorClasses[color as keyof typeof colorClasses]}`}>
          Live
        </span>
      </div>
      
      <h3 className="text-sm font-medium text-gray-600 mb-2">{title}</h3>
      <p className="text-3xl font-bold text-gray-900">{value}</p>
    </div>
  );
}
```

### Sequential Data Fetching

Fetch dependent data sequentially when one request depends on another:

```tsx
// app/users/[id]/page.tsx

async function getUserProfile(userId: string) {
  const response = await fetch(`https://api.example.com/users/${userId}`);
  
  if (!response.ok) {
    throw new Error('Failed to fetch user profile');
  }
  
  return response.json();
}

async function getUserPosts(userId: string) {
  const response = await fetch(`https://api.example.com/users/${userId}/posts`);
  
  if (!response.ok) {
    throw new Error('Failed to fetch user posts');
  }
  
  return response.json();
}

async function getUserComments(userId: string) {
  const response = await fetch(`https://api.example.com/users/${userId}/comments`);
  
  if (!response.ok) {
    throw new Error('Failed to fetch user comments');
  }
  
  return response.json();
}

export default async function UserPage({ params }: { params: { id: string } }) {
  // Sequential: First get user, then fetch posts and comments
  const user = await getUserProfile(params.id);

  // Parallel: Posts and comments are independent, fetch them together
  const [posts, comments] = await Promise.all([
    getUserPosts(params.id),
    getUserComments(params.id),
  ]);

  return (
    <div className="container mx-auto px-4 py-8">
      {/* User profile */}
      <div className="bg-white rounded-lg shadow-md p-6 mb-8">
        <div className="flex items-start space-x-4">
          <div className="relative w-20 h-20 bg-gray-200 rounded-full overflow-hidden">
            <img
              src={user.avatar}
              alt={user.name}
              className="w-full h-full object-cover"
            />
          </div>
          
          <div className="flex-1">
            <h1 className="text-3xl font-bold text-gray-900 mb-2">
              {user.name}
            </h1>
            <p className="text-gray-600 mb-4">{user.bio}</p>
            
            <div className="flex space-x-4 text-sm text-gray-500">
              <span>📧 {user.email}</span>
              <span>📍 {user.location}</span>
              <span>📅 Joined {new Date(user.joinedDate).toLocaleDateString()}</span>
            </div>
          </div>
        </div>
      </div>

      <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
        {/* User posts */}
        <div className="bg-white rounded-lg shadow-md p-6">
          <h2 className="text-xl font-semibold text-gray-900 mb-4">
            Posts ({posts.length})
          </h2>
          
          {posts.length === 0 ? (
            <p className="text-gray-600">No posts yet.</p>
          ) : (
            <div className="space-y-4">
              {posts.map((post: any) => (
                <a
                  key={post.id}
                  href={`/blog/${post.slug}`}
                  className="block group"
                >
                  <h3 className="font-semibold text-gray-900 group-hover:text-blue-600 mb-1">
                    {post.title}
                  </h3>
                  <p className="text-sm text-gray-600 line-clamp-2">
                    {post.excerpt}
                  </p>
                  <div className="flex items-center space-x-4 mt-2 text-xs text-gray-500">
                    <span>👍 {post.likes}</span>
                    <span>💬 {post.comments}</span>
                    <span>📅 {new Date(post.publishedAt).toLocaleDateString()}</span>
                  </div>
                </a>
              ))}
            </div>
          )}
        </div>

        {/* User comments */}
        <div className="bg-white rounded-lg shadow-md p-6">
          <h2 className="text-xl font-semibold text-gray-900 mb-4">
            Comments ({comments.length})
          </h2>
          
          {comments.length === 0 ? (
            <p className="text-gray-600">No comments yet.</p>
          ) : (
            <div className="space-y-4">
              {comments.map((comment: any) => (
                <div key={comment.id} className="border-b border-gray-100 pb-4 last:border-0">
                  <div className="flex items-start space-x-3">
                    <div className="w-8 h-8 bg-gray-200 rounded-full flex items-center justify-center text-sm">
                      {comment.author[0]}
                    </div>
                    <div className="flex-1">
                      <div className="flex items-center justify-between">
                        <span className="font-medium text-gray-900 text-sm">
                          {comment.author}
                        </span>
                        <span className="text-xs text-gray-500">
                          {new Date(comment.createdAt).toLocaleString()}
                        </span>
                      </div>
                      <p className="text-sm text-gray-600 mt-1">{comment.content}</p>
                    </div>
                  </div>
                </div>
              ))}
            </div>
          )}
        </div>
      </div>
    </div>
  );
}
```

### Hybrid Approach

Combine parallel and sequential fetching for optimal performance:

```tsx
// app/products/[id]/page.tsx

async function getProductDetails(productId: string) {
  const response = await fetch(`https://api.example.com/products/${productId}`);
  
  if (!response.ok) {
    throw new Error('Failed to fetch product details');
  }
  
  return response.json();
}

async function getProductReviews(productId: string) {
  const response = await fetch(`https://api.example.com/products/${productId}/reviews`);
  
  if (!response.ok) {
    throw new Error('Failed to fetch product reviews');
  }
  
  return response.json();
}

async function getProductRelatedProducts(productId: string, categoryId: string) {
  const response = await fetch(
    `https://api.example.com/products?category=${categoryId}&exclude=${productId}`
  );
  
  if (!response.ok) {
    throw new Error('Failed to fetch related products');
  }
  
  return response.json();
}

export default async function ProductPage({ params }: { params: { id: string } }) {
  // Sequential: First get product details
  const product = await getProductDetails(params.id);

  // Parallel: Fetch reviews and related products together
  const [reviews, relatedProducts] = await Promise.all([
    getProductReviews(params.id),
    getProductRelatedProducts(params.id, product.categoryId),
  ]);

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="grid grid-cols-1 lg:grid-cols-3 gap-8">
        {/* Main product - 2 columns */}
        <div className="lg:col-span-2 space-y-8">
          <ProductDetails product={product} />
          
          <ProductReviews reviews={reviews} />
        </div>

        {/* Sidebar - 1 column */}
        <div className="space-y-8">
          <RelatedProducts products={relatedProducts} />
        </div>
      </div>
    </div>
  );
}

function ProductDetails({ product }: { product: any }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
        <div className="aspect-square bg-gray-100 rounded-lg overflow-hidden">
          <img
            src={product.imageUrl}
            alt={product.name}
            className="w-full h-full object-cover"
          />
        </div>
        
        <div>
          <h1 className="text-3xl font-bold text-gray-900 mb-4">
            {product.name}
          </h1>
          
          <p className="text-gray-600 mb-6">{product.description}</p>
          
          <div className="mb-6">
            <p className="text-3xl font-bold text-blue-600">
              ${product.price.toFixed(2)}
            </p>
          </div>
          
          <button className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 transition-colors">
            Add to Cart
          </button>
        </div>
      </div>
    </div>
  );
}

function ProductReviews({ reviews }: { reviews: any[] }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-6">Reviews</h2>
      
      {reviews.length === 0 ? (
        <p className="text-gray-600">No reviews yet.</p>
      ) : (
        <div className="space-y-6">
          {reviews.map((review) => (
            <div key={review.id} className="border-b border-gray-100 pb-6 last:border-0">
              <div className="flex items-start space-x-4">
                <div className="w-10 h-10 bg-gray-200 rounded-full flex items-center justify-center">
                  {review.author[0]}
                </div>
                <div className="flex-1">
                  <h3 className="font-semibold text-gray-900">{review.author}</h3>
                  <p className="text-gray-600 mt-2">{review.comment}</p>
                  <p className="text-sm text-gray-500 mt-2">
                    {new Date(review.date).toLocaleDateString()}
                  </p>
                </div>
              </div>
            </div>
          ))}
        </div>
      )}
    </div>
  );
}

function RelatedProducts({ products }: { products: any[] }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-6">Related Products</h2>
      
      <div className="space-y-4">
        {products.map((product) => (
          <a
            key={product.id}
            href={`/products/${product.id}`}
            className="flex items-center space-x-4 group"
          >
            <div className="w-16 h-16 bg-gray-100 rounded overflow-hidden">
              <img
                src={product.thumbnailUrl}
                alt={product.name}
                className="w-full h-full object-cover group-hover:scale-105 transition-transform"
              />
            </div>
            <div className="flex-1">
              <h3 className="font-semibold text-gray-900 group-hover:text-blue-600">
                {product.name}
              </h3>
              <p className="text-gray-600">${product.price.toFixed(2)}</p>
            </div>
          </a>
        ))}
      </div>
    </div>
  );
}
```

## Key Takeaways from Chapter 10

1. **Cache Components and "use cache" Directive**: Cache Components provide granular control over caching at the component level, allowing you to specify caching behavior with `next: { revalidate }` options for different data types and freshness requirements.

2. **Revalidation Strategies**: Implement time-based revalidation with `next: { revalidate }`, stale-while-revalidate patterns for background updates, and on-demand revalidation triggered by application events for precise control over cache updates.

3. **On-Demand Revalidation**: Use `revalidatePath()` and `revalidateTag()` to invalidate cached data after form submissions, external events (webhooks), or scheduled tasks, ensuring data stays fresh without full rebuilds.

4. **Tag-Based Cache Invalidation**: Organize cached data with tags (`tags: ['product-123', 'products']`) and invalidate related data efficiently by revalidating specific tags, making it easy to keep related data in sync.

5. **Incremental Static Regeneration (ISR)**: Combine static generation with dynamic updates using `revalidate`, `generateStaticParams()` for pre-generating pages, and fallback content for failed regenerations, providing both performance and freshness.

6. **Streaming with Suspense**: Use React's `Suspense` component to progressively render page sections as data becomes available, implementing loading skeletons and error boundaries for better perceived performance and user experience.

7. **Parallel and Sequential Data Fetching**: Choose parallel fetching with `Promise.all()` for independent data sources, sequential fetching for dependent data, and hybrid approaches combining both strategies for optimal performance in complex data fetching scenarios.

## Coming Up Next

**Chapter 11: Server Actions**

Now that you've mastered advanced data fetching and caching, it's time to learn about Server Actions, which allow you to run server code directly from your components. In Chapter 11, we'll explore creating Server Actions, handling form submissions, implementing mutation patterns, revalidating after mutations, error handling in Server Actions, and security best practices. This powerful feature simplifies server-side logic and makes building interactive applications easier than ever.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. Core_concepts/9. optimizations_built-in.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='11. server_actions.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
